In [1]:
import ee
from ipyleaflet import TileLayer

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='gee-project-461505')

C:\Users\foson\anaconda3\envs\gee\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(



Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
# GEE에 업로드한 데이터 이름
assetName = "projects/gee-project-461505/assets/KNA_4326_v2"
features = ee.FeatureCollection(assetName)

In [9]:
# 분석 기간
startDate = ee.Date('2024-01-01')
endDate   = ee.Date('2024-12-31')

collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(features)
    .filterDate(startDate, endDate))

labels = collection.select("label")
n_images = collection.size()

class_ids = list(range(9))

# counts 이미지
counts = []
for c in range(9):
    count_c = (collection.map(lambda img, c=c: img.select("label").eq(c))
                 .sum()
                 .rename(f"class_{c}_count"))
    counts.append(count_c)
counts_img = ee.Image.cat(counts)

# majority class 찾기
majority_count = counts_img.select("class_0_count")
majority_id = ee.Image.constant(0)

for c in range(1, 9):
    class_c = counts_img.select(f"class_{c}_count")
    is_bigger = class_c.gt(majority_count)
    majority_count = majority_count.where(is_bigger, class_c)
    majority_id = majority_id.where(is_bigger, c)

majority_class = majority_id.rename("majority_class")
majority_ratio = majority_count.divide(n_images).rename("majority_ratio")

# 최종 결과
result = majority_class.addBands(majority_ratio).clip(features)

NameError: name 'result_clipped' is not defined

In [10]:
# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=result,                   # clip 적용된 이미지
    description='DW_majority_class_ratio',
    folder='GEE_exports',
    fileNamePrefix='dw_majority_class_ratio',
    region=features,                          # 내보낼 범위 (geometry)
    scale=10,                               # 해상도
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    maxPixels=1e13,
    formatOptions={'cloudOptimized': True}
)
task.start()

Export started...
